### Offline/Google COLAB Training script for Bart-Base Baseline Model 
#### Entity-only swaps based of those proposed in Cao et al. 2020 "Factual Error Correction for Abstractive Summarization Models".

All cells should run, however we first need to point to the dataset that is saved in the
the cs287_project folder in Google Drive.

(Uncomment linking to Google Drive Folder if running on COLAB)

In [ ]:
!pip install torch
!pip install transformers datasets rouge_score
import logging
import os
import re
import sys
from dataclasses import dataclass, field
from typing import Optional

import numpy as np
import pandas as pd
import nltk
import torch
import transformers
from datasets import load_dataset, load_metric, load_from_disk
from rouge_score import rouge_scorer
from tqdm import tqdm
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    HfArgumentParser,
    MBartTokenizer,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    default_data_collator,
    set_seed,
)
from transformers.trainer_utils import is_main_process
from datasets import load_dataset

transformers.logging.set_verbosity_info()

DATA_PATH = "data/"

     |████████████████████████████████| 3.1 MB 31.2 MB/s 
     |████████████████████████████████| 298 kB 83.2 MB/s 
     |████████████████████████████████| 596 kB 57.5 MB/s 
     |████████████████████████████████| 61 kB 650 kB/s 
     |████████████████████████████████| 895 kB 72.6 MB/s 
     |████████████████████████████████| 3.3 MB 67.4 MB/s 
     |████████████████████████████████| 132 kB 74.0 MB/s 
     |████████████████████████████████| 243 kB 75.1 MB/s 
     |████████████████████████████████| 1.1 MB 69.2 MB/s 
     |████████████████████████████████| 192 kB 69.7 MB/s 
     |████████████████████████████████| 160 kB 81.6 MB/s 
     |████████████████████████████████| 271 kB 70.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/cs287_project/"

Mounted at /content/drive
/content/drive/My Drive/cs287_project


In [ ]:
!nvidia-smi

Tue Dec  7 16:37:26 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
"""
Load model
"""

MODEL_NAME = "facebook/bart-base"
max_target_length = 128
max_source_length = 1024
padding = False

config = AutoConfig.from_pretrained(
    MODEL_NAME, cache_dir=None, revision="main", use_auth_token=False,
    max_position_embeddings = max_source_length

)

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME, cache_dir=None, use_fast=True, revision="main", use_auth_token=False,
)

model = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_NAME,
    from_tf=bool(".ckpt" in MODEL_NAME),
    config=None,
    cache_dir=None,
    revision="main",
    use_auth_token=False,
)

# Set decoder_start_token_id
if model.config.decoder_start_token_id is None and isinstance(tokenizer, MBartTokenizer):
    model.config.decoder_start_token_id = tokenizer.lang_code_to_id[data_args.target_lang]
if model.config.decoder_start_token_id is None:
    raise ValueError


https://huggingface.co/facebook/bart-base/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpp74snqp_


Downloading:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

storing https://huggingface.co/facebook/bart-base/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/f5310d276a6d1648d00c32fadc8bf7b4607e0fbd5b404fc4a0045960aa2bdfdb.da0f3c0e2dc1c2fecc46738a1ebf4806f2fc36aae3d5c1947f21e063e7cab34b
creating metadata file for /root/.cache/huggingface/transformers/f5310d276a6d1648d00c32fadc8bf7b4607e0fbd5b404fc4a0045960aa2bdfdb.da0f3c0e2dc1c2fecc46738a1ebf4806f2fc36aae3d5c1947f21e063e7cab34b
loading configuration file https://huggingface.co/facebook/bart-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/f5310d276a6d1648d00c32fadc8bf7b4607e0fbd5b404fc4a0045960aa2bdfdb.da0f3c0e2dc1c2fecc46738a1ebf4806f2fc36aae3d5c1947f21e063e7cab34b
Model config BartConfig {
  "_name_or_path": "bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

storing https://huggingface.co/facebook/bart-base/resolve/main/vocab.json in cache at /root/.cache/huggingface/transformers/43978bdeaa326572886b44fcfed82f932f76571095ce31973e51c3da8ccade7f.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
creating metadata file for /root/.cache/huggingface/transformers/43978bdeaa326572886b44fcfed82f932f76571095ce31973e51c3da8ccade7f.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
https://huggingface.co/facebook/bart-base/resolve/main/merges.txt not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpe9fgkfi1


...

storing https://huggingface.co/facebook/bart-base/resolve/main/merges.txt in cache at /root/.cache/huggingface/transformers/3c167ed8af56e6605eeb794b63a79d65d85e6708c9b04408d41946337030f5cd.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
creating metadata file for /root/.cache/huggingface/transformers/3c167ed8af56e6605eeb794b63a79d65d85e6708c9b04408d41946337030f5cd.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
https://huggingface.co/facebook/bart-base/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp70e_i1oe


Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

storing https://huggingface.co/facebook/bart-base/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/a878fcd69bba037c9b1b227f4213579ae43d0aaa9374e167bc6c5f41b1cfeb30.fc9576039592f026ad76a1c231b89aee8668488c671dfbe6616bab2ed298d730
creating metadata file for /root/.cache/huggingface/transformers/a878fcd69bba037c9b1b227f4213579ae43d0aaa9374e167bc6c5f41b1cfeb30.fc9576039592f026ad76a1c231b89aee8668488c671dfbe6616bab2ed298d730
loading file https://huggingface.co/facebook/bart-base/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/43978bdeaa326572886b44fcfed82f932f76571095ce31973e51c3da8ccade7f.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
loading file https://huggingface.co/facebook/bart-base/resolve/main/merges.txt from cache at /root/.cache/huggingface/transformers/3c167ed8af56e6605eeb794b63a79d65d85e6708c9b04408d41946337030f5cd.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
loading file https://hugg

Downloading:   0%|          | 0.00/532M [00:00<?, ?B/s]

storing https://huggingface.co/facebook/bart-base/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/486355ec722ef05fd480e999d4c763be56549ae930f6a3742ee721a5d2a05647.f2f355ad2775769afc60592b43a46d72ca548375e3a1d65f381a751e711cbadd
creating metadata file for /root/.cache/huggingface/transformers/486355ec722ef05fd480e999d4c763be56549ae930f6a3742ee721a5d2a05647.f2f355ad2775769afc60592b43a46d72ca548375e3a1d65f381a751e711cbadd
loading weights file https://huggingface.co/facebook/bart-base/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/486355ec722ef05fd480e999d4c763be56549ae930f6a3742ee721a5d2a05647.f2f355ad2775769afc60592b43a46d72ca548375e3a1d65f381a751e711cbadd
All model checkpoint weights were used when initializing BartForConditionalGeneration.

All the weights of BartForConditionalGeneration were initialized from the model checkpoint at facebook/bart-base.
If your task is similar to the task the model of the checkpoint wa

In [ ]:
# Metric
metric = load_metric("rouge")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    if data_args.ignore_pad_token_for_loss:
        # Replace -100 in the labels as we can't decode them.
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results from ROUGE
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result


Downloading:   0%|          | 0.00/2.17k [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset, load_metric, load_from_disk
train_dataset = load_from_disk("data/xsum_corrupted_entity_untokenized/train")

In [ ]:
dataset = train_dataset.train_test_split(train_size=0.95)
xsum_train = dataset['train']
xsum_eval = dataset['test']

Loading cached split indices for dataset at data/xsum_corrupted_entity_untokenized/train/cache-c06aa4ac5ba461a4.arrow and data/xsum_corrupted_entity_untokenized/train/cache-090f4182897d8e50.arrow


In [ ]:
"""
Load Data
"""

# xsum_train = load_dataset("xsum", split="train")
# xsum_eval = load_dataset("xsum", split="validation")

column_names = xsum_train.column_names
text_column, corrupted_column, target_column = "document", "corrupted_summary", "summary"
# max_source_length = 1024

sep_token = '</s>'
def preprocess_function(examples):

    inputs = examples[text_column]
    corruptions = examples[corrupted_column]
    targets = examples[target_column]

    # Tokenize Input
    model_inputs = tokenizer(
        text = inputs, text_pair=corruptions, max_length=max_source_length, 
        padding=padding, truncation='only_first', add_special_tokens=True
    )
    

    # # Remove EOS tag
    # # model_inputs['input_ids'] =  model_inputs['input_ids'][:-1][:max_source_length-len(model_inputs_summary)] + model_inputs_summary
    # model_inputs['input_ids'] =  model_inputs['input_ids'][:max_source_length-len(model_inputs_summary)] + model_inputs_summary
    # model_inputs['attention_mask'] = [1]*len(model_inputs['input_ids'])
    
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets, max_length=max_target_length, padding=padding, truncation=True
        )

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label]
            for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    
    """
    Uncomment following strings to check if we've encoded properly
    """
    # model_inputs['input_string'] = inputs
    # model_inputs['target_string'] = targets

    return model_inputs

train_dataset = xsum_train
train_dataset = train_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=None,
    remove_columns=column_names,
    load_from_cache_file=False,
)

eval_dataset = xsum_eval
eval_dataset = eval_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=None,
    remove_columns=column_names,
    load_from_cache_file=False,
)

  0%|          | 0/129 [00:00<?, ?ba/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

In [ ]:
debug = False
if debug == True:
  xsum_eval[200]
  len(eval_dataset[200]['input_ids'])
  tokenizer.decode(eval_dataset[200]['input_ids'])
  
  import matplotlib.pyplot as plt
  input_lengths = [len(train_dataset[i]['input_ids']) for i in range(len(train_dataset))] 
  plt.hist(input_lengths)

In [ ]:
from transformers import Seq2SeqTrainingArguments

label_pad_token_id = -100
data_collator = DataCollatorForSeq2Seq(tokenizer, label_pad_token_id=label_pad_token_id)

NUM_GPU = 1
check_val = True

args = Seq2SeqTrainingArguments(
    output_dir="exp/bart_corrupted_entity/results",
    do_train=True,
    do_eval=check_val,
    evaluation_strategy="steps" if check_val else 'no',
    # eval_steps=100000000,
    logging_dir="exp/bart_corrupted_entity/logs",
    num_train_epochs=4,
    logging_steps=1000,
    eval_steps = 5000 if check_val else None,
    # max_steps=int(10000 * 8 / NUM_GPU),
    per_device_train_batch_size=2,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    eval_accumulation_steps=None,
    lr_scheduler_type='polynomial',
    learning_rate=5e-05,
    warmup_steps=500,
    save_steps=20000,
    generation_max_length=64,
)


# Initialize our Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    # compute_metrics=compute_metrics if check_val == True else None
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
# Remove checkpoint if training from scratch
# We used checkpoints since COLAB has limited run-time
trainer.train('exp/bart_corrupted_entity/results/checkpoint-80000')

Loading model from exp/bart_corrupted_entity/results/checkpoint-80000).
***** Running training *****
  Num examples = 128962
  Num Epochs = 4
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 2
  Total optimization steps = 128960
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 2
  Continuing training from global step 80000
  Will skip the first 2 epochs then the first 31040 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command, but you will resume the training on data already seen by your model.


  0%|          | 0/31040 [00:00<?, ?it/s]

Step,Training Loss,Validation Loss
85000,0.047400,0.036783
90000,0.052400,0.036485
95000,0.049500,0.035713
100000,0.043700,0.037269
105000,0.038900,0.035049
110000,0.037000,0.035698
115000,0.041400,0.033891
120000,0.038500,0.033400


***** Running Evaluation *****
  Num examples = 6788
  Batch size = 1
***** Running Evaluation *****
  Num examples = 6788
  Batch size = 1
***** Running Evaluation *****
  Num examples = 6788
  Batch size = 1
***** Running Evaluation *****
  Num examples = 6788
  Batch size = 1
Saving model checkpoint to exp/bart_corrupted_entity/results/checkpoint-100000
Configuration saved in exp/bart_corrupted_entity/results/checkpoint-100000/config.json
Model weights saved in exp/bart_corrupted_entity/results/checkpoint-100000/pytorch_model.bin
tokenizer config file saved in exp/bart_corrupted_entity/results/checkpoint-100000/tokenizer_config.json
Special tokens file saved in exp/bart_corrupted_entity/results/checkpoint-100000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 6788
  Batch size = 1
***** Running Evaluation *****
  Num examples = 6788
  Batch size = 1
***** Running Evaluation *****
  Num examples = 6788
  Batch size = 1
***** Running Evaluation *****
  Num exam

KeyboardInterrupt: ignored

In [ ]:
trainer.save_model()

Saving model checkpoint to exp/bart_corrupted_entity/results
Configuration saved in exp/bart_corrupted_entity/results/config.json
Model weights saved in exp/bart_corrupted_entity/results/pytorch_model.bin
tokenizer config file saved in exp/bart_corrupted_entity/results/tokenizer_config.json
Special tokens file saved in exp/bart_corrupted_entity/results/special_tokens_map.json
